In [ ]:
dart_api1 = '0e3221f05a6a08fbfafbadeaedf57b71f6c74383' #gmail
dart_api2 = '6ff4bb1d83a84b7e471d6a5265510619cb86a5a8' #nate
dart_api3 = '02fe6b29f143c51da3b2489c55f9183da77ea930' #naver
dart_api4 = '019d81f8662b7da731f28b2ebbdb20405621b620' #jh_gmail
dart_api5 = 'e939c83e5ca523aa19e6fc4b7763045dc2d46334' #jh_naver
dart_api6 = '820ae59bd55ca0826e50ac253795ee2acedaa4ed' #jh_gmail2
dart_api7 = 'b11bee34ccc52a704528855b10381b1f58ab1381' #hanmail 
dart_api8 = '09e27a5f2b4a41262e5c28115d74cb11efc76789' #hanmail 
dart_api9 = '6b7c842b3cf2ec4a6b8e4988fa4a668d327d90ff' #gmail2
dart_api10 = '0a13cfae6e5b6ea30c48e29e0cb41100848aa9ce' #gmail3
dart_api11 = '53f014eb00848420936f900073e7cbdb0f2bdf64' #gmail4
dart_api12 = 'd45767c163af35e5c9eb60f1b7eb0eedc6cb3868' #gmail5
dart_api13 = '0394444e2914ef93662a8c962b1ae28107a63af3' #gmail6

In [1]:
import dart_fss as dart
from parse import * 
import pandas as pd
import numpy as np
import sqlalchemy as sql
import sys

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
company_df = pd.read_sql_query(query, sql_engine)

# Open DART API KEY 설정
api_key='0a13cfae6e5b6ea30c48e29e0cb41100848aa9ce'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

def get_value(DF, value_list, number):
    while True:
        for values in value_list:
            result = DF.loc[DF[DF.columns[0]] == values].iloc[:, number]
            if not result.empty:
                break
        break 
    if result.empty:
        return np.nan
    else:
        return result.iloc[0]

class empty_reports:
    report_list = []
    
total_list = []


#for ix in range(778, len(company_df)):
for ix in range(0, len(company_df)): 
    종목코드 = company_df.iloc[ix]['종목코드']
    종목명 = company_df.iloc[ix]['종목명']
    회사정보 = corp_list.find_by_stock_code(종목코드)
    print(ix, 종목코드, 종목명)
    if not 회사정보 == None:
        try:
            reports = 회사정보.search_filings(bgn_de='20181001', end_de='20190831', pblntf_detail_ty=['a001', 'a002', 'a003'], page_count=100)
        except:
            reports = empty_reports()
        for x, y in enumerate(reports.report_list):
            if parse("{}({})", y.report_nm)[1] in ['2019.06', '2019.03', '2018.12', '2018.09']:
                xbrl = reports[x].xbrl
                if not xbrl == None:
                    if xbrl.exist_consolidated():
                        IS = xbrl.get_income_statement(separate=False)
                    else:
                        IS = xbrl.get_income_statement(separate=True)
                    try:
                        DF = IS[0].to_DataFrame(show_class=False, separator=False)
                        print(DF.iloc[:, 0:2])
                        unit_result = parse("{}(Unit: {})", DF.columns[0][0])[1]
                        DF = DF.drop([DF.columns[1], DF.columns[2]], axis =1)
                    except:
                        DF = pd.DataFrame()
                        print(DF)
                    #print(DF)
                    for j in range(1, len(DF.columns)):
                        #print(DF.columns[j])

                        리포트이름 = reports[x].report_nm
                        구분정보 = parse('{}보고서{}', 리포트이름)[0][-2:]
                        구분딕트 = {'사업': '연간', '반기':'2분기누적', '분기':'3분기누적'}

                        회계구분 = DF.columns[j][1][0]
                        회계딕트 = {'연결재무제표' : 'IFRS연결', '별도재무제표': 'KIFRS별도'}

                        기간정보 = DF.columns[j][0]
                        날짜정보 = parse('{}-{}', DF.columns[j][0])
                        시작분기 = pd.to_datetime(날짜정보[0]).quarter
                        년도 = pd.to_datetime(날짜정보[1]).year
                        분기 = pd.to_datetime(날짜정보[1]).quarter

                        if 시작분기 == 분기:
                            분기구분 = '분기'
                        elif 시작분기 != 분기:
                            분기구분 = 구분딕트[구분정보]

                        매출 = get_value(DF, ['ifrs_Revenue', 'ifrs-full_Revenue'], j)
                        영업이익 = get_value(DF, ['dart_OperatingIncomeLoss'], j)
                        당기순이익 = get_value(DF, ['ifrs_ProfitLoss', 'ifrs-full_ProfitLoss'], j)
                        매출총이익 = get_value(DF, ['ifrs_GrossProfit', 'ifrs-full_GrossProfit'], j)
                        지배주주순이익 = get_value(DF, ['ifrs_ProfitLossAttributableToOwnersOfParent', 'ifrs-full_ProfitLossAttributableToOwnersOfParent'], j)

                        result = [종목코드, 종목명, 분기구분, 회계딕트[회계구분], 년도, 분기, unit_result, 기간정보, 매출, 영업이익, 당기순이익, 지배주주순이익, 매출총이익]
                        #print(result)
                        total_list.append(result)

dart_df = pd.DataFrame(total_list, columns= ['종목코드', '종목명', '구분', '회계', '년도', '분기', '단위', '회계기간', '매출액(수익)', '영업이익', '당기순이익', '지배주주순이익', '매출총이익'])
dart_df


Output()

Output()

Output()

ValueError: No axis named 599 for object type DataFrame

In [2]:
print(company_df.iloc[599])


종목코드    016710
종목명      대성홀딩스
Name: 599, dtype: object


In [ ]:
dart_df.insert(loc=0, column='No', value=None)
No_list = []
for i in range(0, len(dart_df)):
    No_list.append(i+1)
dart_df['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(11),
    '회계': sql.types.VARCHAR(11),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '단위': sql.types.VARCHAR(11), 
    '회계기간': sql.types.VARCHAR(50)
}

for i in list(dart_df.columns)[9:]:
    dtype[i] = sql.types.FLOAT() 
    
dart_df.to_sql(name="DART크롤링데이터_rawdata", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `DART크롤링데이터_rawdata` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `DART크롤링데이터_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `DART크롤링데이터_rawdata` ADD INDEX(`종목코드`);')

    

In [ ]:
dart_df
#No col 추가되어있으면 append 완료 
#append 해야함 


In [ ]:
connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')

#dart_df.insert(loc=0, column='No', value=None)

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(11),
    '회계': sql.types.VARCHAR(11),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '단위': sql.types.VARCHAR(11), 
    '회계기간': sql.types.VARCHAR(50)
}

for i in list(dart_df.columns)[9:]:
    dtype[i] = sql.types.FLOAT() 
    
dart_df.to_sql(name="DART크롤링데이터_rawdata", con=sql_engine, if_exists='append', index=False, 
             dtype = dtype)



In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sql
connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT * FROM `DART크롤링데이터_rawdata`"
df = pd.read_sql_query(query, sql_engine)

print(df)
df = df.drop(columns=['No'])
df = df.drop_duplicates()
df

In [ ]:
df.insert(loc=0, column='No', value=None)
No_list = []
for i in range(0, len(df)):
    No_list.append(i+1)
df['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(11),
    '회계': sql.types.VARCHAR(11),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '단위': sql.types.VARCHAR(11), 
    '회계기간': sql.types.VARCHAR(50)
}

for i in list(df.columns)[9:]:
    dtype[i] = sql.types.FLOAT() 
    
df.to_sql(name="DART크롤링데이터_rawdata", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `DART크롤링데이터_rawdata` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `DART크롤링데이터_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `DART크롤링데이터_rawdata` ADD INDEX(`종목코드`);')


In [ ]:
# IS, BS 필요한 항목 가져오기 

import dart_fss as dart
from parse import * 
import pandas as pd
import numpy as np
import sqlalchemy as sql

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
company_df = pd.read_sql_query(query, sql_engine)

# Open DART API KEY 설정
api_key='e939c83e5ca523aa19e6fc4b7763045dc2d46334'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

def get_value(DF, value_list, number):
    while True:
        for values in value_list:
            result = DF.loc[DF[DF.columns[0]] == values].iloc[:, number]
            if not result.empty:
                break
        break 
    if result.empty:
        return np.nan
    else:
        return result.iloc[0]

def get_dict_value(dictionary, key, fin):
    if not key in dictionary.keys():
        if fin == 'IS':
            return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        elif fin == 'BS':
            return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        elif fin == 'CF':
            return [np.nan, np.nan, np.nan]
    else:
        return dictionary[key]

def change_to_DF(table):
    try:
        result_DF = table[0].to_DataFrame(show_class=False, separator=False)
        return result_DF
    except:
        return pd.DataFrame()

class empty_reports:
    report_list = []
    
total_list = []

for ix in range(0 , len(company_df)): 
    종목코드 = company_df.iloc[ix]['종목코드']
    종목명 = company_df.iloc[ix]['종목명']
    회사정보 = corp_list.find_by_stock_code(종목코드)
    print(ix, 종목코드, 종목명)
    if not 회사정보 == None:
        try:
            reports = 회사정보.search_filings(bgn_de='20181001', end_de='20190831', pblntf_detail_ty=['a001', 'a002', 'a003'], page_count=100)
        except:
            reports = empty_reports()
        for x, y in enumerate(reports.report_list):
            if parse("{}({})", y.report_nm)[1] in ['2019.06', '2019.03', '2018.12', '2018.09']:
                print(y.report_nm)
                xbrl = reports[x].xbrl
                if not xbrl == None:
                    if xbrl.exist_consolidated():
                        IS = xbrl.get_income_statement(separate=False)
                        BS = xbrl.get_financial_statement(separate=False)
                        CF = xbrl.get_cash_flows(separate=False)
                    else:
                        IS = xbrl.get_income_statement(separate=True)
                        BS = xbrl.get_financial_statement(separate=True)
                        CF = xbrl.get_cash_flows(separate=True)
                        
                    IS_DF = change_to_DF(IS)
                    BS_DF = change_to_DF(BS)
                    CF_DF = change_to_DF(CF)
                    
                    if not BS_DF.empty:
                        for 필요한항목 in ['투자자산', '유동성장기부채']:
                            if 필요한항목 in list(BS_DF[BS_DF.columns[1]]):
                                print(필요한항목)
                                index_n = BS_DF[BS_DF.columns[1]][BS_DF[BS_DF.columns[1]] == 필요한항목].index[0]
                                print(BS_DF[BS_DF.columns[0]][index_n])
                                
                    if not IS_DF.empty:
                        IS_DF = IS_DF.drop([IS_DF.columns[1], IS_DF.columns[2]], axis =1)
                    if not BS_DF.empty:
                        BS_DF = BS_DF.drop([BS_DF.columns[1], BS_DF.columns[2]], axis =1)
                    if not CF_DF.empty:
                        CF_DF = CF_DF.drop([CF_DF.columns[1], CF_DF.columns[2]], axis =1)
                    
                    if not IS_DF.empty:
                        IS_unit_result = parse("{}(Unit: {})", IS_DF.columns[0][0])[1]
                    else:
                        IS_unit_result = np.nan
                    if not BS_DF.empty:    
                        BS_unit_result = parse("{}(Unit: {})", BS_DF.columns[0][0])[1]
                    else:
                        BS_unit_result = np.nan
                    if not CF_DF.empty:
                        CF_unit_result = parse("{}(Unit: {})", CF_DF.columns[0][0])[1]
                    else:
                        CF_unit_result = np.nan

                    if IS_unit_result == BS_unit_result == CF_unit_result:
                        pass
                    else:
                        print(IS_unit_result, BS_unit_result ,CF_unit_result)
                    
                    IS_dict = {}
                    BS_dict = {}
                    CF_dict = {}

                    for j in range(1, len(IS_DF.columns)):
                        리포트이름 = reports[x].report_nm
                        구분정보 = parse('{}보고서{}', 리포트이름)[0][-2:]
                        구분딕트 = {'사업': '연간', '반기':'2분기누적', '분기':'3분기누적'}

                        회계구분 = IS_DF.columns[j][1][0]
                        회계딕트 = {'연결재무제표' : 'IFRS연결', '별도재무제표': 'KIFRS별도'}

                        #기간정보 = IS_DF.columns[j][0]
                        날짜정보 = parse('{}-{}', IS_DF.columns[j][0])
                        시작분기 = pd.to_datetime(날짜정보[0]).quarter
                        년도 = pd.to_datetime(날짜정보[1]).year
                        분기 = pd.to_datetime(날짜정보[1]).quarter

                        if 시작분기 == 분기:
                            분기구분 = '분기'
                        elif 시작분기 != 분기:
                            분기구분 = 구분딕트[구분정보]

                        매출 = get_value(IS_DF, ['ifrs_Revenue', 'ifrs-full_Revenue'], j)
                        영업이익 = get_value(IS_DF, ['dart_OperatingIncomeLoss'], j)
                        당기순이익 = get_value(IS_DF, ['ifrs_ProfitLoss', 'ifrs-full_ProfitLoss'], j)
                        매출총이익 = get_value(IS_DF, ['ifrs_GrossProfit', 'ifrs-full_GrossProfit'], j)
                        지배주주순이익 = get_value(IS_DF, ['ifrs_ProfitLossAttributableToOwnersOfParent', 'ifrs-full_ProfitLossAttributableToOwnersOfParent'], j)
                        매출원가 = get_value(IS_DF, ['ifrs_CostOfSales', 'ifrs-full_CostOfSales'], j)
                        기본주당이익 = get_value(IS_DF, ['ifrs_BasicEarningsLossPerShare', 'ifrs-full_BasicEarningsLossPerShare'], j)
                        희석주당이익 = get_value(IS_DF, ['ifrs_DilutedEarningsLossPerShare', 'ifrs-full_DilutedEarningsLossPerShare'], j)
                        IS_dict[(분기구분, 회계딕트[회계구분], 년도, 분기, IS_unit_result)] = [매출, 매출원가, 매출총이익, 영업이익, 당기순이익,지배주주순이익, 기본주당이익, 희석주당이익]
                        
                    for j in range(1, len(BS_DF.columns)):
                        리포트이름 = reports[x].report_nm
                        구분정보 = parse('{}보고서{}', 리포트이름)[0][-2:]
                        구분딕트 = {4: '연간', 2:'2분기누적', 3:'3분기누적'}

                        회계구분 = BS_DF.columns[j][1][0]
                        회계딕트 = {'연결재무제표' : 'IFRS연결', '별도재무제표': 'KIFRS별도'}

                        기간정보 = BS_DF.columns[j][0]
                        년도 = pd.to_datetime(기간정보).year
                        분기 = pd.to_datetime(기간정보).quarter

                        if 분기 == 1:
                            분기구분 = '분기'
                        else: 
                            분기구분 = 구분딕트[분기]
                            
                        자산총계 = get_value(BS_DF, ['ifrs_Assets', 'ifrs-full_Assets'], j)
                        유동자산 = get_value(BS_DF, ['ifrs_CurrentAssets', 'ifrs-full_CurrentAssets'], j)
                        재고자산 = get_value(BS_DF, ['ifrs_Inventories', 'ifrs-full_Inventories'], j)
                        투자자산 = get_value(BS_DF, ['ifrs-full_InvestmentsInSubsidiariesJointVenturesAndAssociates', 'ifrs_InvestmentsInSubsidiariesJointVenturesAndAssociates', 'ifrs_InvestmentProperty', 'ifrs-full_InvestmentProperty'], j)
                        유형자산 = get_value(BS_DF, ['ifrs_PropertyPlantAndEquipment', 'ifrs-full_PropertyPlantAndEquipment'], j)

                        부채총계 = get_value(BS_DF, ['ifrs_Liabilities', 'ifrs-full_Liabilities'], j)
                        유동부채 = get_value(BS_DF, ['ifrs_CurrentLiabilities', 'ifrs-full_CurrentLiabilities'], j)
                        장기차입금 = get_value(BS_DF, ['dart_LongTermBorrowingsGross'], j)
                        차입금 = get_value(BS_DF, ['ifrs_ShorttermBorrowings', 'ifrs-full_ShorttermBorrowings'], j)
                        사채 = get_value(BS_DF, ['dart_BondsIssued'], j)
                        유동성장기부채 = get_value(BS_DF, ['ifrs-full_CurrentPortionOfLongtermBorrowings', 'ifrs_CurrentPortionOfLongtermBorrowings'], j) #, 'entity00101628_udf_BS_2020115165953478_CurrentLiabilities'

                        자본총계 = get_value(BS_DF, ['ifrs_Equity', 'ifrs-full_Equity'], j)
                        지배주주자본 = get_value(BS_DF, ['ifrs_EquityAttributableToOwnersOfParent', 'ifrs-full_EquityAttributableToOwnersOfParent'], j)
                        매출채권및기타채권 = get_value(BS_DF, ['ifrs_TradeAndOtherCurrentReceivables', 'ifrs-full_TradeAndOtherCurrentReceivables'], j)
                        매출채권 = get_value(BS_DF, ['dart_ShortTermTradeReceivable'], j)
                        
                        #주식수 = get_value(DF, '', j)
                        
                        if 분기 == 1:
                            BS_dict[('분기', 회계딕트[회계구분], 년도, 분기, BS_unit_result)] = [자산총계, 유동자산, 재고자산, 투자자산, 유형자산, 부채총계, 유동부채, 장기차입금, 차입금, 사채, 유동성장기부채, 자본총계, 지배주주자본, 매출채권및기타채권, 매출채권]
                        else:
                            BS_dict[(분기구분, 회계딕트[회계구분], 년도, 분기, BS_unit_result)] = [자산총계, 유동자산, 재고자산, 투자자산, 유형자산, 부채총계, 유동부채, 장기차입금, 차입금, 사채, 유동성장기부채, 자본총계, 지배주주자본, 매출채권및기타채권, 매출채권]
                            BS_dict[('분기', 회계딕트[회계구분], 년도, 분기, BS_unit_result)] = [자산총계, 유동자산, 재고자산, 투자자산, 유형자산, 부채총계, 유동부채, 장기차입금, 차입금, 사채, 유동성장기부채, 자본총계, 지배주주자본, 매출채권및기타채권, 매출채권]
                        
                        
                    for j in range(1, len(CF_DF.columns)):
                        리포트이름 = reports[x].report_nm
                        구분정보 = parse('{}보고서{}', 리포트이름)[0][-2:]
                        구분딕트 = {'사업': '연간', '반기':'2분기누적', '분기':'3분기누적'}

                        회계구분 = CF_DF.columns[j][1][0]
                        회계딕트 = {'연결재무제표' : 'IFRS연결', '별도재무제표': 'KIFRS별도'}

                        #기간정보 = CF_DF.columns[j][0]
                        날짜정보 = parse('{}-{}', CF_DF.columns[j][0])
                        if not 날짜정보 == None:
                            시작분기 = pd.to_datetime(날짜정보[0]).quarter
                            년도 = pd.to_datetime(날짜정보[1]).year
                            분기 = pd.to_datetime(날짜정보[1]).quarter
                        else:
                            시작분기 = '모름'
                            년도 = pd.to_datetime(CF_DF.columns[j][0]).year
                            분기 = pd.to_datetime(CF_DF.columns[j][0]).quarter
                            
                        if 시작분기 == 분기:
                            분기구분 = '분기'
                        elif 시작분기 == '모름':
                            분기구분 = '확인필요'
                        elif 시작분기 != 분기:
                            분기구분 = 구분딕트[구분정보]
                        
                        영업활동현금흐름 = get_value(CF_DF, ['ifrs_CashFlowsFromUsedInOperatingActivities', 'ifrs-full_CashFlowsFromUsedInOperatingActivities'], j)
                        투자활동현금흐름 = get_value(CF_DF, ['ifrs_CashFlowsFromUsedInInvestingActivities', 'ifrs-full_CashFlowsFromUsedInInvestingActivities'], j)
                        재무활동현금흐름 = get_value(CF_DF, ['ifrs_CashFlowsFromUsedInFinancingActivities', 'ifrs-full_CashFlowsFromUsedInFinancingActivities'], j)

                        CF_dict[(분기구분, 회계딕트[회계구분], 년도, 분기, CF_unit_result)] = [영업활동현금흐름, 투자활동현금흐름, 재무활동현금흐름]
                    
                    date_set = set(list(IS_dict.keys()) + list(BS_dict.keys()) + list(CF_dict.keys()))
                    for x in date_set:
                        result = [종목코드, 종목명]+list(x)+get_dict_value(IS_dict, x, 'IS')+get_dict_value(BS_dict, x, 'BS')+get_dict_value(CF_dict, x, 'CF')
                        total_list.append(result)



In [ ]:
dart_df = pd.DataFrame(total_list, columns= ['종목코드', '종목명', '구분', '회계', '년도', '분기', '단위', 
                                             '매출액', '매출원가', '매출총이익', '영업이익', '당기순이익','지배주주순이익', '기본주당이익', '희석주당이익',
                                             '자산총계', '유동자산', '재고자산', '투자자산', '유형자산', '부채총계', '유동부채', '장기차입금', '차입금', '사채', '유동성장기부채', '자본총계', '지배주주자본', '매출채권및기타채권', '매출채권',  
                                            '영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름'])
dart_df


In [ ]:
dart_df.insert(loc=0, column='No', value=None)

"""No_list = []
for i in range(0, len(dart_df)):
    No_list.append(i+1)
dart_df['No'] = No_list"""

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(11),
    '회계': sql.types.VARCHAR(11),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '단위': sql.types.VARCHAR(11)
}

for i in list(dart_df.columns)[9:]:
    dtype[i] = sql.types.FLOAT() 

dart_df.to_sql(name="DART크롤링데이터_2020년3분기", con=sql_engine, if_exists='append', index=False, 
             dtype = dtype)

"""dart_df.to_sql(name="DART크롤링데이터_2020년3분기", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `DART크롤링데이터_2020년3분기` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `DART크롤링데이터_2020년3분기` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `DART크롤링데이터_2020년3분기` ADD INDEX(`종목코드`);')"""

In [53]:
# IS, BS 필요한 항목 가져오기 

import dart_fss as dart
from parse import * 
import pandas as pd
import numpy as np
import sqlalchemy as sql

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
company_df = pd.read_sql_query(query, sql_engine)

# Open DART API KEY 설정
api_key='e939c83e5ca523aa19e6fc4b7763045dc2d46334'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

def get_value(DF, value_list, number):
    while True:
        for values in value_list:
            result = DF.loc[DF[DF.columns[0]] == values].iloc[:, number]
            if not result.empty:
                break
        break 
    if result.empty:
        return np.nan
    else:
        return result.iloc[0]

def get_dict_value(dictionary, key, fin):
    if not key in dictionary.keys():
        if fin == 'IS':
            return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        elif fin == 'BS':
            return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
    else:
        return dictionary[key]

def change_to_DF(table):
    try:
        result_DF = table[0].to_DataFrame(show_class=False, separator=False)
        return result_DF
    except:
        return pd.DataFrame()

class empty_reports:
    report_list = []
    
total_list = []

for ix in range(0 , len(company_df)): 
    종목코드 = company_df.iloc[ix]['종목코드']
    종목명 = company_df.iloc[ix]['종목명']
    회사정보 = corp_list.find_by_stock_code(종목코드)
    #print(ix, 종목코드, 종목명)
    if not 회사정보 == None:
        try:
            reports = 회사정보.search_filings(bgn_de='20181001', pblntf_detail_ty=['a001', 'a002', 'a003'], page_count=100)
            pop_list = []
            for rpts in reports.report_list:
                if '기재정정' in rpts.report_nm:
                    타겟분기 = parse("{}({})", rpts.report_nm)[1]
                    for index,rptss in enumerate(reports.report_list):
                        if '기재정정' not in rptss.report_nm and (parse("{}({})", rptss.report_nm)[1] == 타겟분기):
                            pop_list.append(index)
            pop_list.sort(reverse=True)
            for pop in pop_list:
                reports.pop(index=pop)
        except:
            reports = empty_reports()
            
        for x, y in enumerate(reports.report_list):
            선택날짜 = parse("{}({})", y.report_nm)[1]
            if 선택날짜 in ['2020.09', '2020.06', '2020.03', '2019.12', '2019.09', '2019.06', '2019.03', '2018.12', '2018.09']:
                #print(y.report_nm)
                xbrl = reports[x].xbrl
                if not xbrl == None:
                    if xbrl.exist_consolidated():
                        IS = xbrl.get_income_statement(separate=False)
                        BS = xbrl.get_financial_statement(separate=False)
                        IS_DF = change_to_DF(IS)
                        BS_DF = change_to_DF(BS)
                        if not IS_DF.empty:
                            if 선택날짜 in ['2020.09', '2019.09', '2018.09']:
                                IS_DF = IS_DF[[IS_DF.columns[0], IS_DF.columns[3], IS_DF.columns[4], IS_DF.columns[5], IS_DF.columns[6]]]
                            elif 선택날짜 == '2018.12':
                                IS_DF = IS_DF.drop([IS_DF.columns[1], IS_DF.columns[2]], axis =1)
                            else:
                                IS_DF = IS_DF[[IS_DF.columns[0], IS_DF.columns[3], IS_DF.columns[4]]]
                        if not BS_DF.empty:
                            BS_DF = BS_DF[[BS_DF.columns[0], BS_DF.columns[3], BS_DF.columns[4]]]
                    else:
                        IS = xbrl.get_income_statement(separate=True)
                        BS = xbrl.get_financial_statement(separate=True)
                        IS_DF = change_to_DF(IS)
                        BS_DF = change_to_DF(BS)
                        if not IS_DF.empty:
                            if 선택날짜 in ['2020.09', '2019.09', '2018.09']:
                                IS_DF = IS_DF[[IS_DF.columns[0], IS_DF.columns[3], IS_DF.columns[4]]]
                            elif 선택날짜 == '2018.12':
                                IS_DF = IS_DF.drop([IS_DF.columns[1], IS_DF.columns[2]], axis =1)
                            else:
                                IS_DF = IS_DF[[IS_DF.columns[0], IS_DF.columns[3]]]
                        if not BS_DF.empty:
                            BS_DF = BS_DF[[BS_DF.columns[0], BS_DF.columns[3]]]
                    
                    if not IS_DF.empty:
                        IS_unit_result = parse("{}(Unit: {})", IS_DF.columns[0][0])[1]
                    else:
                        IS_unit_result = np.nan
                    if not BS_DF.empty:    
                        BS_unit_result = parse("{}(Unit: {})", BS_DF.columns[0][0])[1]
                    else:
                        BS_unit_result = np.nan

                    if IS_unit_result == BS_unit_result:
                        pass
                    else:
                        print(IS_unit_result, BS_unit_result)
                    
                    IS_dict = {}
                    BS_dict = {}

                    for j in range(1, len(IS_DF.columns)):
                        리포트이름 = reports[x].report_nm
                        구분정보 = parse('{}보고서{}', 리포트이름)[0][-2:]
                        구분딕트 = {'사업': '연간', '반기':'2분기누적', '분기':'3분기누적'}

                        회계구분 = IS_DF.columns[j][1][0]
                        회계딕트 = {'연결재무제표' : 'IFRS연결', '별도재무제표': 'KIFRS별도'}

                        #기간정보 = IS_DF.columns[j][0]
                        날짜정보 = parse('{}-{}', IS_DF.columns[j][0])
                        시작분기 = pd.to_datetime(날짜정보[0]).quarter
                        년도 = pd.to_datetime(날짜정보[1]).year
                        분기 = pd.to_datetime(날짜정보[1]).quarter

                        if 시작분기 == 분기:
                            분기구분 = '분기'
                        elif 시작분기 != 분기:
                            분기구분 = 구분딕트[구분정보]

                        매출 = get_value(IS_DF, ['ifrs_Revenue', 'ifrs-full_Revenue'], j)
                        영업이익 = get_value(IS_DF, ['dart_OperatingIncomeLoss'], j)
                        당기순이익 = get_value(IS_DF, ['ifrs_ProfitLoss', 'ifrs-full_ProfitLoss'], j)
                        매출총이익 = get_value(IS_DF, ['ifrs_GrossProfit', 'ifrs-full_GrossProfit'], j)
                        지배주주순이익 = get_value(IS_DF, ['ifrs_ProfitLossAttributableToOwnersOfParent', 'ifrs-full_ProfitLossAttributableToOwnersOfParent'], j)
                        매출원가 = get_value(IS_DF, ['ifrs_CostOfSales', 'ifrs-full_CostOfSales'], j)
                        IS_dict[(분기구분, 회계딕트[회계구분], 년도, 분기, IS_unit_result)] = [매출, 매출원가, 매출총이익, 영업이익, 당기순이익,지배주주순이익]
                        
                    for j in range(1, len(BS_DF.columns)):
                        리포트이름 = reports[x].report_nm
                        구분정보 = parse('{}보고서{}', 리포트이름)[0][-2:]
                        구분딕트 = {4: '연간', 2:'2분기누적', 3:'3분기누적'}

                        회계구분 = BS_DF.columns[j][1][0]
                        회계딕트 = {'연결재무제표' : 'IFRS연결', '별도재무제표': 'KIFRS별도'}

                        기간정보 = BS_DF.columns[j][0]
                        년도 = pd.to_datetime(기간정보).year
                        분기 = pd.to_datetime(기간정보).quarter

                        if 분기 == 1:
                            분기구분 = '분기'
                        else: 
                            분기구분 = 구분딕트[분기]
                            
                        자산총계 = get_value(BS_DF, ['ifrs_Assets', 'ifrs-full_Assets'], j)
                        유동자산 = get_value(BS_DF, ['ifrs_CurrentAssets', 'ifrs-full_CurrentAssets'], j)
                        재고자산 = get_value(BS_DF, ['ifrs_Inventories', 'ifrs-full_Inventories'], j)
                        투자자산 = get_value(BS_DF, ['ifrs-full_InvestmentsInSubsidiariesJointVenturesAndAssociates', 'ifrs_InvestmentsInSubsidiariesJointVenturesAndAssociates', 'ifrs_InvestmentProperty', 'ifrs-full_InvestmentProperty'], j)
                        
                        부채총계 = get_value(BS_DF, ['ifrs_Liabilities', 'ifrs-full_Liabilities'], j)
                        유동부채 = get_value(BS_DF, ['ifrs_CurrentLiabilities', 'ifrs-full_CurrentLiabilities'], j)
                        
                        자본총계 = get_value(BS_DF, ['ifrs_Equity', 'ifrs-full_Equity'], j)
                        지배주주자본 = get_value(BS_DF, ['ifrs_EquityAttributableToOwnersOfParent', 'ifrs-full_EquityAttributableToOwnersOfParent'], j)
                        매출채권 = get_value(BS_DF, ['dart_ShortTermTradeReceivable'], j)
                        
                        if 분기 == 1 or 분기 == 2:
                            BS_dict[('분기', 회계딕트[회계구분], 년도, 분기, BS_unit_result)] = [자산총계, 유동자산, 재고자산, 투자자산, 부채총계, 유동부채, 자본총계, 지배주주자본, 매출채권]
                        elif 분기 ==3:
                            BS_dict[(분기구분, 회계딕트[회계구분], 년도, 분기, BS_unit_result)] = [자산총계, 유동자산, 재고자산, 투자자산, 부채총계, 유동부채, 자본총계, 지배주주자본, 매출채권]
                            BS_dict[('분기', 회계딕트[회계구분], 년도, 분기, BS_unit_result)] = [자산총계, 유동자산, 재고자산, 투자자산, 부채총계, 유동부채, 자본총계, 지배주주자본, 매출채권]
                        else:
                            BS_dict[(분기구분, 회계딕트[회계구분], 년도, 분기, BS_unit_result)] = [자산총계, 유동자산, 재고자산, 투자자산, 부채총계, 유동부채, 자본총계, 지배주주자본, 매출채권]
                                                    
                    date_set = set(list(IS_dict.keys()) + list(BS_dict.keys()))
                    for x in date_set:
                        result = [종목코드, 종목명]+list(x)+get_dict_value(IS_dict, x, 'IS')+get_dict_value(BS_dict, x, 'BS')
                        total_list.append(result)

dart_df = pd.DataFrame(total_list, columns= ['종목코드', '종목명', '구분', '회계', '년도', '분기', '단위', 
                                             '매출액', '매출원가', '매출총이익', '영업이익', '당기순이익','지배주주순이익',
                                             '자산총계', '유동자산', '재고자산', '투자자산', '부채총계', '유동부채', '자본총계', '지배주주자본', '매출채권'])
dart_df.insert(loc=0, column='No', value=None)
No_list = []
for i in range(0, len(dart_df)):
    No_list.append(i+1)
dart_df['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(11),
    '회계': sql.types.VARCHAR(11),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '단위': sql.types.VARCHAR(11)
}

for i in list(dart_df.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 
    
dart_df.to_sql(name="DART_크롤링데이터_rawdata", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `DART_크롤링데이터_rawdata` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `DART_크롤링데이터_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `DART_크롤링데이터_rawdata` ADD INDEX(`종목코드`);')

print("DART_크롤링데이터_rawdata 크롤링 완료")

500 011300 성안
분기보고서 (2020.09)


Output()

Output()

/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


반기보고서 (2020.06)


...

Output()

분기보고서 (2018.09)


Output()

Output()

,종목코드,종목명,구분,회계,년도,분기,단위,매출액,매출원가,매출총이익,...,지배주주순이익,자산총계,유동자산,재고자산,투자자산,부채총계,유동부채,자본총계,지배주주자본,매출채권
0,011300,성안,분기,KIFRS별도,2020,3,KRW,4596454120.0,3968178751.0,628275369.0,...,nan,138253120093.0,29311256187.0,7408070939.0,13516568793.0,74790002090.0,45705979332.0,63463118003.0,nan,nan
1,011300,성안,분기,IFRS연결,2020,3,KRW,25028945482.0,28171371063.0,-3142425581.0,...,-6290614238.0,289313045332.0,138019018872.0,68432721769.0,13884445733.0,227238688001.0,163621978057.0,62074357331.0,51401507098.0,nan
2,011300,성안,3분기누적,KIFRS별도,2020,3,KRW,25923937562.0,23138111004.0,2785826558.0,...,nan,138253120093.0,29311256187.0,7408070939.0,13516568793.0,74790002090.0,45705979332.0,63463118003.0,nan,nan
3,011300,성안,3분기누적,IFRS연결,2020,3,KRW,97017024753.0,98193952533.0,-1176927780.0,...,-13515022767.0,289313045332.0,138019018872.0,68432721769.0,13884445733.0,227238688001.0,163621978057.0,62074357331.0,51401507098.0,nan
4,011300,성안,2분기누적,KIFRS별도,2020,2,KRW,nan,nan,nan,...,nan,138951712422.0,34005573439.0,7279241075.0,13595922425.0,74209473921.0,44095513410.0,64742238501.0,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,011390,부산산업,연간,KIFRS별도,2016,4,KRW,32206767107.0,26746734773.0,5460032334.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
158,011390,부산산업,분기,KIFRS별도,2018,3,KRW,7953642957.0,6477806970.0,1475835987.0,...,nan,47885454717.0,12645957474.0,447155051.0,nan,10559289845.0,7568517551.0,37326164872.0,nan,nan
159,011390,부산산업,분기,IFRS연결,2018,3,KRW,25843925319.0,22740637941.0,3103287378.0,...,253385352.0,125748119732.0,56288412615.0,25519109885.0,nan,55138970636.0,40343552014.0,70609149096.0,51685124245.0,nan
160,011390,부산산업,3분기누적,KIFRS별도,2018,3,KRW,24649158085.0,20149524295.0,4499633790.0,...,nan,47885454717.0,12645957474.0,447155051.0,nan,10559289845.0,7568517551.0,37326164872.0,nan,nan


In [54]:
dart_df.insert(loc=0, column='No', value=None)
No_list = []
for i in range(0, len(dart_df)):
    No_list.append(i+1)
dart_df['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(11),
    '회계': sql.types.VARCHAR(11),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '단위': sql.types.VARCHAR(11), 
    '회계기간': sql.types.VARCHAR(50)
}

for i in list(dart_df.columns)[9:]:
    dtype[i] = sql.types.FLOAT() 
    
dart_df.to_sql(name="DART_크롤링데이터_rawdata", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `DART_크롤링데이터_rawdata` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `DART_크롤링데이터_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `DART_크롤링데이터_rawdata` ADD INDEX(`종목코드`);')

    